In [2]:
from datetime import datetime, timezone
from pyspark.sql import SparkSession, Row
from pyspark.sql.functions import lit
import sys

date = '2020-01-01'

spark = (SparkSession
         .builder
         .config('hive.metastore.uris', 'thrift://localhost:9083')
         .enableHiveSupport()
         .getOrCreate())

def parse_line(line):
    f = line.split()
    wbanno = f[0]
    dt = datetime.strptime(f[1] + f[2], '%Y%m%d%H%M')
    dt = dt.replace(tzinfo=timezone.utc)
    temperature = None if f[8] == '-9999.0' else float(f[8])
    return Row(timestamp=dt, wbanno=wbanno, temperature=temperature)

rdd = spark.sparkContext.textFile(f"s3a://datalake/uscrn/{date}/*")
df = rdd.map(parse_line).toDF()
df = df.withColumn('year', lit(date[:4]))
df.write.partitionBy('year').saveAsTable('uscrn', mode='overwrite')

spark.sql('show tables').show()

Py4JJavaError: An error occurred while calling o45.partitions.
: java.lang.RuntimeException: java.lang.ClassNotFoundException: Class org.apache.hadoop.fs.s3a.S3AFileSystem not found
	at org.apache.hadoop.conf.Configuration.getClass(Configuration.java:2197)
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:2654)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:2667)
	at org.apache.hadoop.fs.FileSystem.access$200(FileSystem.java:94)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:2703)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:2685)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:373)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:295)
	at org.apache.hadoop.mapred.FileInputFormat.singleThreadedListStatus(FileInputFormat.java:258)
	at org.apache.hadoop.mapred.FileInputFormat.listStatus(FileInputFormat.java:229)
	at org.apache.hadoop.mapred.FileInputFormat.getSplits(FileInputFormat.java:315)
	